In [2]:
import os, sys
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)




In [3]:
from langchain_tavily import TavilySearch


In [4]:
from langchain.tools import tool
from langchain.vectorstores import FAISS
from app.rag.embeddings import get_embedder

# Build absolute path to FAISS index using PROJECT_ROOT from Cell 1
vector_db_path = os.path.join(PROJECT_ROOT, "artifacts", "vector_db", "agri_faiss_index")
print(f"Resolved FAISS path: {vector_db_path}")

embedder = get_embedder()
vectorstore = None
if os.path.exists(vector_db_path):
    vectorstore = FAISS.load_local(
        vector_db_path,
        embedder,
        allow_dangerous_deserialization=True,
    )
    print("FAISS Index Loaded")
else:
    print(f"FAISS Index not found at {vector_db_path}")


Resolved FAISS path: c:\Users\JAYADIR\OneDrive\Desktop\major_proj\artifacts\vector_db\agri_faiss_index
🛡️ OOM Protection: Forcing CPU mode for Embeddings.
⏳ Loading Jina Embeddings V3 on cpu...


`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


✅ Jina V3 Loaded Successfully
FAISS Index Loaded
FAISS Index Loaded


In [5]:
from pydantic import BaseModel, Field
from langchain.tools import tool

class RetrievalArgs(BaseModel):
    query: str = Field(..., description="User query to search local vector store")

@tool("retrieval_tool", args_schema=RetrievalArgs)
def retrieval_tool(query: str) -> str:
    """Retrieve top-5 documents from the local FAISS vector store based on the query."""
    if "vectorstore" not in globals() or vectorstore is None:
        return "[retrieval_tool] Local FAISS index is not loaded; no documents available."
    docs = vectorstore.similarity_search(query, k=5)
    return "\n\n".join(doc.page_content for doc in docs)


In [6]:
from langgraph.graph import StateGraph, END
from langgraph.nodes import ToolNode

ModuleNotFoundError: No module named 'langgraph.nodes'

In [11]:
from langgraph.prebuilt import create_react_agent
from langchain_tavily import TavilySearch
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool as lc_tool
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv
load_dotenv()
memory=MemorySaver()
model=ChatGroq(
    model="openai/gpt-oss-20b"
)
# Initialize Tavily Search Tool
_tavily = TavilySearch(
    max_results=5,
    topic="general",
    include_raw_content=True
)

@lc_tool("web_search")
def web_search(query: str) -> str:
    """Web search via Tavily (topic forced to 'general'). Returns titles + URLs."""
    result = _tavily.invoke({"query": query, "topic": "general"})
    try:
        return "\n".join(
            f"- {item.get('title','No title')} — {item.get('url','')}" for item in (result or [])
        )
    except Exception:
        return str(result)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an agriculture expert assistant. You are provided with 2 tools: one is a retriever which can fetch you documents related to pests, and the other is a web search tool. ALWAYS FIRST USE THE retrieval_tool. If you are not satisfied with the results from the retriever, then use the web search tool to get more information. Always provide references for the information you provide to the user.**IMPORTANT: REMEMBER ONE MOST IMPORTANT THING, REPLY ONLY IN PLAIN TEXT, DONT USE ANY FORMATTING AND MARKDOWN**"),
    ("human", "{user_query}")
])
messages = prompt.format_messages(user_query="what is meant by rabi?")

agent = create_react_agent(model, [web_search, retrieval_tool],checkpointer=memory)
config = {"configurable": {"thread_id": "test_thread_001"}}
for step in agent.stream(
    {"messages": messages},
    stream_mode="values",
    config=config
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

what is meant by rabi?
================================== Ai Message ==================================
Tool Calls:
  retrieval_tool (fc_538346e7-56b4-4686-947d-d5975c9b1f2d)
 Call ID: fc_538346e7-56b4-4686-947d-d5975c9b1f2d
  Args:
    query: rabi season definition
================================== Ai Message ==================================
Tool Calls:
  retrieval_tool (fc_538346e7-56b4-4686-947d-d5975c9b1f2d)
 Call ID: fc_538346e7-56b4-4686-947d-d5975c9b1f2d
  Args:
    query: rabi season definition
================================= Tool Message =================================
Name: retrieval_tool

Cabbage moth: S. et al.

Cabbage moth: The head is a copper color. At the sixth and final instar, the head remains copper and the body and develop a dorsal hump

Africanized bee: It appears that cows sustain hundreds of stings if they are attacked, but can survive such injuries.

Rat: The term "lab rat"

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `openai/gpt-oss-20b` in organization `org_01jbvze0qze9ysgqgs5pz1x6x5` service tier `on_demand` on tokens per minute (TPM): Limit 8000, Requested 11188, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [9]:
state=agent.get_state(config=config)
print(state.values.get("messages"))

[SystemMessage(content='You are an agriculture expert assistant. You are provided with 2 tools: one is a retriever which can fetch you documents related to pests, and the other is a web search tool. ALWAYS FIRST USE THE retrieval_tool. If you are not satisfied with the results from the retriever, then use the web search tool to get more information. Always provide references for the information you provide to the user.', additional_kwargs={}, response_metadata={}, id='f96b0f12-e03b-41e6-83a5-adc86e0c1f95'), HumanMessage(content='what is the best crop to grow n telangana in winter?', additional_kwargs={}, response_metadata={}, id='7fc47104-d2a8-4ad4-ac2c-213185256207'), AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to answer: best crop to grow in Telangana in winter. Provide references. Use retriever first.', 'tool_calls': [{'id': 'fc_43ae3390-b398-4343-822d-a9680f3c16b0', 'function': {'arguments': '{"query":"best crop to grow in Telangana in winter"}', 'name': 

In [12]:
raw_input="""
# Significance of Rabi season\n\nRabi season is a crucial cropping period in both India and Pakistan. In India, it generally lasts from October to March and is suitable for growing various crops, including tomatoes. Similarly, in Pakistan, the Rabi season is a major agricultural period primarily used for cultivating crops like wheat, with analyses focusing on its flows and impact. The season\'s timing and importance highlight its significance in the agricultural landscapes of both nations.\n\nSynonyms: [Winter growing season](/concept/winter-growing-season "Explore winter growing season concept and significance"), [Post-monsoon season](/concept/post-monsoon-season "Explore post-monsoon season concept and significance"), [Winter crop](/concept/winter-crop "Explore winter crop concept and significance"), [Spring harvest](/concept/spring-harvest "Explore spring harvest concept and significance")\n\nThe below excerpts are indicatory and do represent direct quotations or translations. It is your responsibility to fact check each reference.\n\n## The concept of Rabi season in scientific sources\n\n[Science](/science/concepts/ "View all Science concepts") [Books](/science/ "View all Science books and articles")\n\nRabi season is a crucial cropping period in India and Pakistan, spanning from October to March. It\'s suitable for growing crops like tomatoes and wheat, with analysis of its flows being important in Pakistan.\n\nSignificance in Environmental Sciences:\n\n[Environmental Sciences](/environmental-sciences/concepts/ "View all Environmental Sciences concepts") [Books](/environmental-sciences/ "View all Environmental Sciences books and articles")\n\nFrom: [*Sustainability Journal (MDPI)*](/science/journal/sustainability-journal-mdpi)\n\n(1) The Rabi Season, occurring from December to April, shows mean values of 100 mm and 77 mm for SEBAL-based ETa and advection aridity, respectively, according to the text.[[1]](/science/journal/sustainability-journal-mdpi/d/doc1796765.html "Reference: Evaluating the Spatio-Temporal Distribution of Irrigation Water Components...")  (2) Rabi seasons are the winter cropping seasons, and the two-year experiment was carried out in the Instructional Unit, SKN Agricultural University, Jobner, during the rabi seasons of 2014–2015 and 2015–2016.[[2]](/science/journal/sustainability-journal-mdpi/d/doc1829321.html "Reference: Effect of Irrigation Schedule and Organic Fertilizer on Wheat Yield, Nutrient...")  (3) It is a season that accounted for the maximum cropped area, followed by the kharif and summer seasons, indicating its significance in agricultural production.[[3]](/science/journal/sustainability-journal-mdpi/d/doc1848734.html "Reference: Multi-Temporal Analysis of Cropping Patterns and Intensity Using Optical and...")  (4) The Rabi season experiences adverse incidences in weather parameters like relative humidity and unseasonal rainfall, impacting crop yields.[[4]](/science/journal/sustainability-journal-mdpi/d/doc1797411.html "Reference: Can a Weather-Based Crop Insurance Scheme Increase the Technical Efficiency...")  (5) It is the winter or dry season in India, typically from November to March, during which crops like pulses and oilseeds are grown, often utilizing residual soil moisture.[[5]](/science/journal/sustainability-journal-mdpi/d/doc1805854.html "Reference: Energy Balance, Productivity and Resource-Use Efficiency of Diverse...")'}, {'url': 'https://juagrisciences.com/blog/Crop/Rabi-Season-A-Season-of-Prosperity-for-Farmers', 'title': 'Rabi Season: A Season of Prosperity for Farmers', 'content': "In this blog, we will explore the significance of the Rabi season and how it benefits farmers through crop production. Key Crops of the Rabi Season a. Diversified Income: The cultivation of different crops during the Rabi season diversifies farmers' income sources. d. Enhanced Food Security: The production of staple crops like wheat and barley during the Rabi season contributes significantly to India's food security. The Rabi season is a crucial period in India's agricultural calendar, bringing prosperity to farmers through the cultivation of various crops. While challenges exist, the benefits of the Rabi season make it an essential part of India's agriculture, contributing significantly to the nation's economy and the well-being of its farmers.", 'score': 0.78709817, 'raw_content': "* [Crop](https://juagrisciences.com/blog/category/Crop)\n\n# [Rabi Season: A Season of Prosperity for Farmers](#)\n\nIndia’s diverse climate gives birth to various agricultural seasons, each with its unique significance. One of these seasons is the Rabi Season, which typically falls from November to April. Rabi, which means “spring” in Arabic, is aptly named as it corresponds to the winter months when crops are sown and harvested during the spring season. This season plays a pivotal role in India’s agricultural calendar, and farmers greatly benefit from the cultivation of various crops during this time. In this blog, we will explore the significance of the Rabi season and how it benefits farmers through crop production.\n\nKey Crops of the Rabi Season\n\nThe Rabi season is known for the cultivation of various crops that thrive in the cooler months. Some of the key crops grown during this season include:\n\na. Wheat: Wheat is the most prominent Rabi crop in India and is a staple food for millions. States like Punjab, Haryana, Uttar Pradesh, and Madhya Pradesh are major wheat-producing regions. The Rabi season provides the ideal conditions for wheat cultivation, resulting in a significant boost to India's food security.\n\nb. Barley: Barley is another vital Rabi crop grown in regions with cooler climates. It is used for various purposes, including food, fodder, and the brewing industry. Rajasthan, Uttar Pradesh, and Madhya Pradesh are major barley-producing states.\n\nc. Mustard: Mustard is a key oilseed crop cultivated during the Rabi season. It provides edible oil and is grown in states like Rajasthan, Uttar Pradesh, and Madhya Pradesh.\n\nd. Gram (Chickpeas): Chickpeas are rich in protein and a popular pulse crop during the Rabi season. They are grown in states like Madhya Pradesh, Rajasthan, and Uttar Pradesh.\n\ne. Lentils: Lentils are another important pulse crop, providing a valuable source of protein. They are cultivated in various Rabi-producing states across India.\n\nBenefits to Farmers\n\nThe Rabi season brings several benefits to farmers, making it a crucial part of India's agricultural calendar:\n\na. Diversified Income: The cultivation of different crops during the Rabi season diversifies farmers' income sources. This helps reduce the risk associated with mono-cropping and provides a more stable financial foundation.\n\nb. Improved Soil Health: Rabi crops often have deep root systems, which help improve soil structure and nutrient content. This enhances the fertility of the soil for subsequent seasons.\n\nc. Water Utilization: Rabi crops typically require less water than Kharif crops, making efficient use of available water resources. This is especially important in regions with limited water availability.\n\nd. Enhanced Food Security: The production of staple crops like wheat and barley during the Rabi season contributes significantly to India's food security. It ensures a consistent supply of essential grains throughout the year.\n\ne. Livestock Feed: Some Rabi crops, like barley and chickpeas, are used as fodder for livestock, supporting the dairy and animal husbandry sectors.\n\nf. Crop Rotation: Rabi crops fit well into crop rotation cycles, reducing the risk of pests and diseases and improving overall crop health.\n\nChallenges Faced by Farmers\n\nWhile the Rabi season offers numerous benefits, farmers also face challenges, including:\n\na. Weather Variability: Unpredictable weather patterns, including untimely rainfall or frost, can damage Rabi crops.\n\nb. Input Costs: The cost of inputs like seeds, fertilizers, and pesticides can be a burden on farmers, affecting their profitability.\n\nc. Market Price Fluctuations: Farmers are often vulnerable to price fluctuations in the market, which can impact their income.\n\nThe Rabi season is a crucial period in India's agricultural calendar, bringing prosperity to farmers through the cultivation of various crops. It not only supports food security but also enhances soil health, diversifies income sources, and promotes efficient water use. While challenges exist, the benefits of the Rabi season make it an essential part of India's agriculture, contributing significantly to the nation's economy and the well-being of its farmers.\n\n## Categories\n\n* [Farming16](https://juagrisciences.com/blog/category/Farming)\n* [Crop8](https://juagrisciences.com/blog/category/Crop)\n* [Farmer's Stories3](https://juagrisciences.com/blog/category/Farmer-s-Stories)\n\n## Related Blogs\n\n### [Ayaka—The Patented Innovation Setting New Standards in Farming](https://juagrisciences.com/blog/Crop/Ayaka-The-Patented-Innovation-Setting-New-Standards-in-Farming)\n\n15/06/2025\n\n### [Saffron: The Labor-Intensive Spice That Defines Luxury](https://juagrisciences.com/blog/Crop/Saffron-The-Labor-Intensive-Spice-That-Defines-Luxury)\n\n20/02/2025\n\n### [Ultimate Guide to Wheat Production in India](https://juagrisciences.com/blog/Crop/Ultimate-Guide-to-Wheat-Production-in-India)\n\n18/03/2024\n\n### [Nutrient Management for Rice Crops: Balancing Fertilization for Maximum Yield](https://juagrisciences.com/blog/Crop/Nutrient-Management-for-Rice-Crops-Balancing-Fertilization-for-Maximum-Yield)\n\n28/02/2024\n\n### [Good Agricultural Practices: Cultivating a Sustainable Future](https://juagrisciences.com/blog/Crop/Good-Agricultural-Practices-Cultivating-a-Sustainable-Future)\n\n### [Top 5 Most Profitable Crops in India](https://juagrisciences.com/blog/Crop/Top-5-Most-Profitable-Crops-in-India)\n\n### [Rabi Season: A Season of Prosperity for Farmers](https://juagrisciences.com/blog/Crop/Rabi-Season-A-Season-of-Prosperity-for-Farmers)\n\n06/10/2023\n\n### [Farming Foundation: The Science of Crop Selection](https://juagrisciences.com/blog/Crop/Farming-Foundation-The-Science-of-Crop-Selection)\n\n  \n "}, {'url': 'https://en.wikipedia.org/wiki/Rabi_crop', 'title': 'Rabi crop', 'content': '# Rabi crop. Classification of crops harvested in spring season in the Indian Subcontinent. **Rabi crops** or the **rabi harvest**, also known as **winter crops**, are agricultural crops that are sown in winter and harvested in the spring in India, Pakistan, and Bangladesh. Complementary to the rabi crop is the kharif crop, which is grown after the rabi and zaid crops are harvested one after another respectively. The term is derived from the Arabic word for "spring "Spring (season)")", which is used in the Indian subcontinent, where it is the spring harvest (also known as the "winter crop"). The major rabi crop in India is wheat, followed by barley, mustard, sesame and peas. Many crops are cultivated in both kharif and rabi seasons. ## Common rabi crops. * tomato (*Solanum lycopersiucum, L*) (tamatar). * India: 2003/04 Rabi Crop Assessment, US Department of Agriculture.', 'score': 0.69672287, 'raw_content': '[Jump to content](#bodyContent)\n\n[Search](/wiki/Special:Search "Search Wikipedia [f]")\n\n## Contents\n\n* [(Top)](#)\n* [1 Etymology](#Etymology)\n* [2 Rabi season in India](#Rabi_season_in_India)\n* [3 Common rabi crops](#Common_rabi_crops)\n  + [3.1 Cereals](#Cereals)\n  + [3.2 Fruits](#Fruits)\n  + [3.3 Legumes / lentils (dal)](#Legumes_/_lentils_(dal))\n  + [3.4 Seed plants](#Seed_plants)\n  + [3.5 Vegetables](#Vegetables)\n  + [3.6 Others](#Others)\n* [4 See also](#See_also)\n* [5 References](#References)\n* [6 External links](#External_links)\n\n# Rabi crop\n\n* [বাংলা](https://bn.wikipedia.org/wiki/%E0%A6%B0%E0%A6%AC%E0%A6%BF%E0%A6%B6%E0%A6%B8%E0%A7%8D%E0%A6%AF "রবিশস্য – Bangla")\n* [भोजपुरी](https://bh.wikipedia.org/wiki/%E0%A4%B0%E0%A4%AC%E0%A5%80 "रबी – Bhojpuri")\n* [Deutsch](https://de.wikipedia.org/wiki/Rabi-Feldfr%C3%BCchte "Rabi-Feldfrüchte – German")\n* [Français](https://fr.wikipedia.org/wiki/Culture_rabi "Culture rabi – French")\n* [ગુજરાતી](https://gu.wikipedia.org/wiki/%E0%AA%B0%E0%AA%B5%E0%AA%BF_%E0%AA%AA%E0%AA%BE%E0%AA%95 "રવિ પાક – Gujarati")\n* [हिन्दी](https://hi.wikipedia.org/wiki/%E0%A4%B0%E0%A4%AC%E0%A5%80_%E0%A4%95%E0%A5%80_%E0%A4%AB%E0%A4%BC%E0%A4%B8%E0%A4%B2 "रबी की फ़सल – Hindi")\n* [മലയാളം](https://ml.wikipedia.org/wiki/%E0%B4%B1%E0%B4%AC%E0%B4%BF_%E0%B4%B5%E0%B4%BF%E0%B4%B3 "റബി വിള – Malayalam")\n* [ਪੰਜਾਬੀ](https://pa.wikipedia.org/wiki/%E0%A8%B0%E0%A8%AC%E0%A9%80_%E0%A8%A6%E0%A9%80_%E0%A8%AB%E0%A8%BC%E0%A8%B8%E0%A8%B2 "ਰਬੀ ਦੀ ਫ਼ਸਲ – Punjabi")\n* [پنجابی](https://pnb.wikipedia.org/wiki/%DB%81%D8%A7%DA%91%DA%BE%DB%8C "ہاڑھی – Western Punjabi")\n* [Simple English](https://simple.wikipedia.org/wiki/Rabi_crops "Rabi crops – Simple English")\n* [Suomi](https://fi.wikipedia.org/wiki/Rabi_(vilja) "Rabi (vilja) – Finnish")\n* [தமிழ்](https://ta.wikipedia.org/wiki/%E0%AE%95%E0%AF%81%E0%AE%B1%E0%AF%81%E0%AE%B5%E0%AF%88_%E0%AE%9A%E0%AE%BE%E0%AE%95%E0%AF%81%E0%AE%AA%E0%AE%9F%E0%AE%BF "குறுவை சாகுபடி – Tamil")\n* [తెలుగు](https://te.wikipedia.org/wiki/%E0%B0%B0%E0%B0%AC%E0%B1%80_%E0%B0%AA%E0%B0%82%E0%B0%9F "రబీ పంట – Telugu")\n\n[Edit links](https://www.wikidata.org/wiki/Special:EntityPage/Q3631367#sitelinks-wikipedia "Edit interlanguage links")\n\n* [Article](/wiki/Rabi_crop "View the content page [c]")\n* [Talk](/wiki/Talk:Rabi_crop "Discuss improvements to the content page [t]")\n\n* [Read](/wiki/Rabi_crop)\n* [Edit](/w/index.php?title=Rabi_crop&action=edit "Edit this page [e]")\n* [View history](/w/index.php?title=Rabi_crop&action=history "Past revisions of this page [h]")\n\n\n\nTools\n\nActions\n\n* [Read](/wiki/Rabi_crop)\n* [Edit](/w/index.php?title=Rabi_crop&action=edit "Edit this page [e]")\n* [View history](/w/index.php?title=Rabi_crop&action=history)\n\nGeneral\n\n* [What links here](/wiki/Special:WhatLinksHere/Rabi_crop "List of all English Wikipedia pages containing links to this page [j]")\n* [Related changes](/wiki/Special:RecentChangesLinked/Rabi_crop "Recent changes in pages linked from this page [k]")\n* [Upload file](//en.wikipedia.org/wiki/Wikipedia:File_Upload_Wizard "Upload files [u]")\n* [Permanent link](/w/index.php?title=Rabi_crop&oldid=1319898656 "Permanent link to this revision of this page")\n* [Page information](/w/index.php?title=Rabi_crop&action=info "More information about this page")\n* [Cite this page](/w/index.php?title=Special:CiteThisPage&page=Rabi_crop&id=1319898656&wpFormIdentifier=titleform "Information on how to cite this page")\n* [Get shortened URL](/w/index.php?title=Special:UrlShortener&url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FRabi_crop)\n* [Download QR code](/w/index.php?title=Special:QrCode&url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FRabi_crop)\n\nPrint/export\n\n* [Download as PDF](/w/index.php?title=Special:DownloadAsPdf&page=Rabi_crop&action=show-download-screen "Download this page as a PDF file")\n* [Printable version](/w/index.php?title=Rabi_crop&printable=yes "Printable version of this page [p]")\n\nIn other projects\n\n* [Wikidata item](https://www.wikidata.org/wiki/Special:EntityPage/Q3631367 "Structured data on this page hosted by Wikidata [g]")\n\nAppearance\n\n**Checked**\n\nFrom Wikipedia, the free encyclopedia\n\n## Page version status\n\nThis is an accepted version of this page\n\nThis is the [latest accepted revision](/wiki/Wikipedia:Pending_changes "Wikipedia:Pending changes"), [reviewed](https://en.wikipedia.org/w/index.php?title=Special:Log&type=review&page=Rabi_crop) on *1 November 2025*.\n\nClassification of crops harvested in spring season in the Indian Subcontinent\n\n|  |\n| --- |\n| This article **needs additional citations for [verification](/wiki/Wikipedia:Verifiability "Wikipedia:Verifiability")**. Please help [improve this article](/wiki/Special:EditPage/Rabi_crop "Special:EditPage/Rabi crop") by [adding citations to reliable sources](/wiki/Help:Referencing_for_beginners "Help:Referencing for beginners"). Unsourced material may be challenged and removed. *Find sources:*\xa0["Rabi crop"](https://www.google.com/search?as_eq=wikipedia&q=%22Rabi+crop%22)\xa0–\xa0[news](https://www.google.com/search?tbm=nws&q=%22Rabi+crop%22+-wikipedia&tbs=ar:1)\xa0**·** [newspapers](https://www.google.com/search?&q=%22Rabi+crop%22&tbs=bkt:s&tbm=bks)\xa0**·** [books](https://www.google.com/search?tbs=bks:1&q=%22Rabi+crop%22+-wikipedia)\xa0**·** [scholar](https://scholar.google.com/scholar?q=%22Rabi+crop%22)\xa0**·** [JSTOR](https://www.jstor.org/action/doBasicSearch?Query=%22Rabi+crop%22&acc=on&wc=on) *(March 2019)* *([Learn how and when to remove this message](/wiki/Help:Maintenance_template_removal "Help:Maintenance template removal"))* |\n\n**Rabi crops** or the **rabi harvest**, also known as **winter crops**, are [agricultural crops](/wiki/Crop "Crop") that are sown in [winter](/wiki/Winter "Winter") and harvested in the spring in [India](/wiki/India "India"), [Pakistan](/wiki/Pakistan "Pakistan"), and [Bangladesh](/wiki/Bangladesh "Bangladesh").[[1]](#cite_note-1) Complementary to the rabi crop is the [kharif crop](/wiki/Kharif_crop "Kharif crop"), which is grown after the rabi and [zaid crops](/wiki/Zaid_crop "Zaid crop") are harvested one after another respectively.\n\nThe word rabi was borrowed from [Persian](/wiki/Persian_language "Persian language"): ربيع, [romanized](/wiki/Romanization_of_Persian "Romanization of Persian"):\xa0*rabī*, [lit.](/wiki/Literal_translation "Literal translation")\u2009\'spring\'.\n\n## Etymology\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=1 "Edit section: Etymology")]\n\nThe words *kharif* and *rabi* have their origins in Arabic. These came to be used in India with the ascent of the [Mughal empire](/wiki/Mughal_Empire "Mughal Empire") in the Indian subcontinent and have been widely used ever since. The term is derived from the [Arabic](/wiki/Arabic "Arabic") word for "[spring](/wiki/Spring_(season) "Spring (season)")", which is used in the [Indian subcontinent](/wiki/Indian_subcontinent "Indian subcontinent"), where it is the spring harvest (also known as the "winter crop").\n\n## Rabi season in India\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=2 "Edit section: Rabi season in India")]\n\nRabi crops are sown around mid-November, preferably after the [monsoon](/wiki/Monsoon "Monsoon") rains are over, and harvesting begins in April / May. The crops are grown either with rainwater that has [percolated](/wiki/Percolation "Percolation") into the ground or using [irrigation](/wiki/Irrigation "Irrigation"). Good rain in winter spoils rabi crops but is good for [kharif crops](/wiki/Kharif_crops "Kharif crops").\n\nThe major rabi crop in India is [wheat](/wiki/Wheat "Wheat"), followed by [barley](/wiki/Barley "Barley"), [mustard](/wiki/Mustard_seed "Mustard seed"), [sesame](/wiki/Sesame "Sesame") and [peas](/wiki/Peas "Peas"). Peas are harvested early, as they are ready early: Indian markets are flooded with green peas from January to March, peaking in February.\n\nMany crops are cultivated in both kharif and rabi seasons. The crops produced in India are seasonal and highly dependent on these two monsoons.[[2]](#cite_note-2) The table below contains a list of differences between the three cropping seasons in India.\n\nThe Indian government also offers Minimum Support Price for these crops, so that the farmers can benefit from the harvest.[[3]](#cite_note-3)\n\n## Common rabi crops\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=3 "Edit section: Common rabi crops")]\n\n### Cereals\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=4 "Edit section: Cereals")]\n\n* [barley](/wiki/Barley "Barley")\n* [oat](/wiki/Oat "Oat") (*Avena sativa*)\n* [Wheat](/wiki/Wheat "Wheat")\n\n### Fruits\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=5 "Edit section: Fruits")]\n\nList as follows. These are rabi harvests rather than crops as that term is usually applied to [annuals](/wiki/Annual_plant "Annual plant") and not [perennials](/wiki/Perennial_plant "Perennial plant"):[[4]](#cite_note-rab1-4)\n\n* [banana](/wiki/Banana "Banana")\n* [ber](/wiki/Ber "Ber")\n* [date](/wiki/Date_palm "Date palm")\n* [grape](/wiki/Grape "Grape")\n* [grape fruit](/wiki/Grape_fruit "Grape fruit")\n* [guava](/wiki/Guava "Guava")\n* [kinnow](/wiki/Kinnow "Kinnow")\n* [lemon](/wiki/Lemon "Lemon")\n* [lime](/wiki/Lime_(fruit) "Lime (fruit)")\n* [mandarin orange](/wiki/Mandarin_orange "Mandarin orange")\n* [mangoes](/wiki/Mango "Mango")\n* [mulberries](/wiki/Morus_(plant) "Morus (plant)")\n* [orange](/wiki/Orange_(fruit) "Orange (fruit)")\n\n### Legumes / lentils ([dal](/wiki/Dal "Dal"))\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=6 "Edit section: Legumes / lentils (dal)")]\n\n* [chickpea](/wiki/Chickpea "Chickpea")[[5]](#cite_note-chic1-5)\n* [kulthi](/wiki/Macrotyloma_uniflorum "Macrotyloma uniflorum") (horse gram)[[5]](#cite_note-chic1-5)\n* [lobhia](/wiki/Black-eyed_pea "Black-eyed pea")[[5]](#cite_note-chic1-5)\n* [masoor](/wiki/Lentil#Types "Lentil")[[5]](#cite_note-chic1-5)\n* [pigeon pea](/wiki/Pigeon_pea "Pigeon pea")\n\n### Seed plants\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=7 "Edit section: Seed plants")]\n\n* [alfalfa](/wiki/Alfalfa "Alfalfa") (also known as lucerne, *Medicago sativa*)\n* [coriander](/wiki/Coriander "Coriander") (*Coriandrum sativum, L*)\n* [cumin](/wiki/Cumin "Cumin") (*Cuminum cyminum, L*)\n* [fenugreek](/wiki/Fenugreek "Fenugreek") (*Trigonella foenumgraecum, L*)\n* [linseed](/wiki/Linseed "Linseed")\n* [mustard](/wiki/Mustard_plant "Mustard plant") (*Brassica juncea L.*)\n* [isabgol](/wiki/Plantago_ovata "Plantago ovata") (*Plantago ovata*)\n* [sunflower](/wiki/Sunflower "Sunflower")\n* [Bengal gram](/wiki/Chickpea "Chickpea")\n* [red gram](/wiki/Red_gram "Red gram")\n* [black pepper](/wiki/Black_pepper "Black pepper")\n\n### Vegetables\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=8 "Edit section: Vegetables")]\n\n* [bean](/wiki/Bean "Bean")\n* [beetroot](/wiki/Beetroot "Beetroot") (chukundar)\n* [brinjal](/wiki/Brinjal "Brinjal") (baingan)\n* [broccoli](/wiki/Broccoli "Broccoli") (hari gobhi)\n* [cabbage](/wiki/Cabbage "Cabbage") (patta gobhi)\n* [capsicum](/wiki/Capsicum "Capsicum") (shimla mirch) [[6]](#cite_note-rab2-6)\n* [carrot](/wiki/Carrot "Carrot") (gajar)\n* [cauliflowers](/wiki/Cauliflowers "Cauliflowers") (gobhi)\n* [chickpea](/wiki/Chickpea "Chickpea") (also known as gram, *Cicer arientinum*) (chana)\n* [fenugreek](/wiki/Fenugreek "Fenugreek") (methi)\n* [garlic](/wiki/Garlic "Garlic") (lehsun)\n* [lady finger](/wiki/Okra "Okra") (bhindi)\n* [lettuce](/wiki/Lettuce "Lettuce") (salad gobhi)\n* [pea](/wiki/Pea "Pea") (matar)\n* [onion](/wiki/Onion "Onion") (*Allium cepa, L.*) (pyaj)\n* [potato](/wiki/Potato "Potato") (*Solanum tuberosum*) (urulai kizhangu) (aloo)\n* [radish](/wiki/Radish "Radish") (mooli)\n* [spinach](/wiki/Spinach "Spinach") (palak)\n* [sweet potato](/wiki/Sweet_potato "Sweet potato") (shakarkand)\n* [tomato](/wiki/Tomato "Tomato") (*Solanum lycopersiucum, L*) (tamatar)\n* [turnip](/wiki/Turnip "Turnip") (shalgam)\n\n### Others\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=9 "Edit section: Others")]\n\n* [tobacco](/wiki/Tobacco "Tobacco")[[4]](#cite_note-rab1-4)\n\n## See also\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=10 "Edit section: See also")]\n\n* [Zaid crops](/wiki/Zaid_crops "Zaid crops"), a minor cropping season (summer) in the Indian Subcontinent\n* [Kharif crop](/wiki/Kharif_crop "Kharif crop"), a major cropping season based on the [Arriving Monsoon](/wiki/Monsoon "Monsoon") in the Indian Subcontinent\n* [Cash crop](/wiki/Cash_crop "Cash crop")\n* [Winter wheat](/wiki/Winter_wheat "Winter wheat"), a crop grown in winter in colder climates\n\n## References\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=11 "Edit section: References")]\n\n1. **[^](#cite_ref-1)** Balfor, Edward (1885). [*The Cyclopaedia of India and of Eastern and Southern Asia*](https://archive.org/stream/cyclopindiaeast03balfuoft#page/331/mode/1up) (3\xa0ed.). London: Bernard Quaritch. p.\xa0331. [Archived](https://web.archive.org/web/20140415184601/https://archive.org/stream/cyclopindiaeast03balfuoft#page/331/mode/1up) from the original on 2014-04-15.\n2. **[^](#cite_ref-2)** [Sowing time of Rabi & Kharif crop | agropedia](http://agropedia.iitk.ac.in/content/sowing-time-rabi-kharif-crop) [Archived](https://web.archive.org/web/20130105051529/http://agropedia.iitk.ac.in/content/sowing-time-rabi-kharif-crop) 2013-01-05 at the [Wayback Machine](/wiki/Wayback_Machine "Wayback Machine")\n3. **[^](#cite_ref-3)** [Revised MSP notification](https://desagri.gov.in/notification-type/rabi/), Released by the Directorate of Economics and Statistics under the Ministry of Agriculture\n4. ^ [***a***](#cite_ref-rab1_4-0) [***b***](#cite_ref-rab1_4-1) [Rabi products list](http://agrinfobank.com.pk/rabi-crops-vegetables-fruits-and-pulses/), released by Agrinfobank, Pakistan\n5. ^ [***a***](#cite_ref-chic1_5-0) [***b***](#cite_ref-chic1_5-1) [***c***](#cite_ref-chic1_5-2) [***d***](#cite_ref-chic1_5-3) [Rabi crop planting rises 10% in a week](https://economictimes.indiatimes.com/news/economy/agriculture/wheat-sowing-up-7-rice-cereals-acreage-down-so-far/articleshow/56022773.cms), 2016.\n6. **[^](#cite_ref-rab2_6-0)** [Rabi crops in Haryana](http://haryanaseeds.gov.in/products_vegetables_rabi.html), Haryana Seeds Development Corporation.\n\n## External links\n\n[[edit](/w/index.php?title=Rabi_crop&action=edit&section=12 "Edit section: External links")]\n\n* [India: 2003/04 Rabi Crop Assessment](https://web.archive.org/web/20070610052510/http://www.fas.usda.gov/pecad2/highlights/2004/04/India2004Assess/index.htm), US Department of Agriculture\n\n[Portals](/wiki/Wikipedia:Contents/Portals "Wikipedia:Contents/Portals"):\n\n* [Agriculture](/wiki/Portal:Agriculture "Portal:Agriculture")\n* [Food](/wiki/Portal:Food "Portal:Food")\n* [Weather](/wiki/Portal:Weather "Portal:Weather")\n\nRetrieved from "<https://en.wikipedia.org/w/index.php?title=Rabi_crop&oldid=1319898656>"\n\n[Categories](/wiki/Help:Category "Help:Category"):\n\n* [Crops](/wiki/Category:Crops "Category:Crops")\n* [Agriculture in India](/wiki/Category:Agriculture_in_India "Category:Agriculture in India")\n* [Agriculture in Pakistan](/wiki/Category:Agriculture_in_Pakistan "Category:Agriculture in Pakistan")\n\nHidden categories:\n\n* [Webarchive template wayback links](/wiki/Category:Webarchive_template_wayback_links "Category:Webarchive template wayback links")\n* [Articles with short description](/wiki/Category:Articles_with_short_description "Category:Articles with short description")\n* [Short description is different from Wikidata](/wiki/Category:Short_description_is_different_from_Wikidata "Category:Short description is different from Wikidata")\n* [Wikipedia pending changes protected pages](/wiki/Category:Wikipedia_pending_changes_protected_pages "Category:Wikipedia pending changes protected pages")\n* [Articles needing additional references from March 2019](/wiki/Category:Articles_needing_additional_references_from_March_2019 "Category:Articles needing additional references from March 2019")\n* [All articles needing additional references](/wiki/Category:All_articles_needing_additional_references "Category:All articles needing additional references")\n* [Articles containing Persian-language text](/wiki/Category:Articles_containing_Persian-language_text "Category:Articles containing Persian-language text")\n\nRabi crop\n\n[Add topic](#)\n\n '}, {'url': 'https://www.doubtnut.com/pcmb-questions/127714', 'title': 'Write short note on Rabi crops.', 'content': 'Rabi crops or Rabi harvests are the crops in agriculture that are sown in winter or cold season. They are harvested in the spring.Read more', 'score': 0.6796259, 'raw_content': None}, {'url': 'https://www.kisaanhelpline.com/rabi-crops', 'title': 'Rabi Crops List - Example of Rabi Season Crops in India', 'content': 'Career Login Register # Major Rabi Crops Indian rabi crops or rabi fasal are rural crops that are sown in the winter season in India and harvested in the spring. In Indian farming, Rabi crops include wheat, barley, oats (cereals), gram/chana (pulses), linseed, garlic, cumin, coriander, mustard (oilseeds), etc. Most of the Rabi crops including wheat, barley, oats (cereals), gram/chana (pulses), linseed, and mustard are sown in winter in India and harvested in spring. In the type of Rabi crop, we can divide it into some categories like food crops, fodder crops, fiber crops, oil crops, ornamental crops and industrial crops. ###### Rabi Onion (रबी प्याज) * Indian Crop * Agri Updates * Agri Research * Login', 'score': 0.5412882, 
"""

In [14]:
import re

def clean_scraped_text(text):
    """
    Step 1: Aggressive cleaning of specific scraped artifacts.
    """
    if not text: return ""
    
    # 1. Remove Markdown links [Text](URL) -> Text
    text = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', text)
    
    # 2. Remove Citations [1], [a], [note 1]
    text = re.sub(r'\[\d+\]|\[note \d+\]|\[[a-z]\]', '', text)
    
    # 3. Remove Wiki Headers/Edit buttons (e.g., "[edit]", "Jump to")
    text = re.sub(r'\[edit\]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'(Jump to content|Edit section:.*|Edit links)', '', text, flags=re.IGNORECASE)
    
    # 4. Remove Table artifacts (e.g., "| --- |", "|")
    text = re.sub(r'\|.*\|', '', text) 
    
    # 5. Remove specific "Noise" lines common in scraping
    noise_keywords = [
        "Retrieved from", "Hidden categories", "Webarchive template",
        "Download as PDF", "Printable version", "Permanent link",
        "Tools", "Actions", "General", "Appearance", "Checked", "Wikipedia"
    ]
    
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        line = line.strip()
        # Skip empty or noise lines
        if len(line) < 3: continue 
        if any(keyword.lower() in line.lower() for keyword in noise_keywords): continue
        
        cleaned_lines.append(line)
        
    return cleaned_lines # Returns a list of strings, not a single string yet

def merge_paragraphs(lines):
    """
    Step 2: The 'One-Word Chunk' Fix.
    Merges fragmented lines into cohesive paragraphs.
    """
    merged_paragraphs = []
    current_para = []

    for line in lines:
        # Detect if this line is a Header or Bullet point (start of new block)
        is_header = line.startswith('#')
        is_bullet = line.startswith('*') or line.startswith('-') or line.startswith('+')
        
        if is_header or is_bullet:
            # If we were building a paragraph, save it first
            if current_para:
                merged_paragraphs.append(" ".join(current_para))
                current_para = []
            # Add the header/bullet as its own distinct block
            merged_paragraphs.append(line)
        else:
            # It's regular text, add to current paragraph buffer
            current_para.append(line)
            
    # Append any remaining text
    if current_para:
        merged_paragraphs.append(" ".join(current_para))
        
    return merged_paragraphs

def create_smart_chunks(paragraphs, max_chars=1000, min_chars=100):
    """
    Step 3: Group paragraphs into chunks. 
    Enforces a minimum size to avoid tiny chunks.
    """
    chunks = []
    current_chunk = ""
    
    for para in paragraphs:
        # If adding this paragraph exceeds max size, save current chunk
        if len(current_chunk) + len(para) > max_chars:
            if len(current_chunk) >= min_chars:
                chunks.append(current_chunk.strip())
                current_chunk = para # Start new chunk with current para
            else:
                # If current chunk is too small, force append (or handle differently)
                current_chunk += " " + para
        else:
            # Add to current chunk
            current_chunk += "\n\n" + para if current_chunk else para
            
    # Add the final chunk
    if current_chunk:
        chunks.append(current_chunk.strip())
        
    return chunks

# --- Execution ---



# 1. Clean
clean_lines = clean_scraped_text(raw_input)

# 2. Merge (Fixes the fragmentation)
coherent_paragraphs = merge_paragraphs(clean_lines)

# 3. Chunk
final_chunks = create_smart_chunks(coherent_paragraphs, max_chars=800, min_chars=50)

# Output results
for i, chunk in enumerate(final_chunks):
    print(f"--- CHUNK {i+1} ({len(chunk)} chars) ---\n{chunk}\n")

--- CHUNK 1 (307 chars) ---
# Significance of Rabi season

Synonyms: Winter growing season, Post-monsoon season, Winter crop, Spring harvest The below excerpts are indicatory and do represent direct quotations or translations. It is your responsibility to fact check each reference.

## The concept of Rabi season in scientific sources

--- CHUNK 2 (2953 chars) ---
Science Books Rabi season is a crucial cropping period in India and Pakistan, spanning from October to March. It's suitable for growing crops like tomatoes and wheat, with analysis of its flows being important in Pakistan. Significance in Environmental Sciences: Environmental Sciences Books From: *Sustainability Journal (MDPI)* (1) The Rabi Season, occurring from December to April, shows mean values of 100 mm and 77 mm for SEBAL-based ETa and advection aridity, respectively, according to the text.[](/science/journal/sustainability-journal-mdpi/d/doc1796765.html "Reference: Evaluating the Spatio-Temporal Distribution of Irrigat

In [20]:
import torch
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    "pszemraj/long-t5-tglobal-base-16384-book-summary",
    device=0 if torch.cuda.is_available() else -1,
)
long_text = """# Rabi Season: A Season of Prosperity for Farmers India’s diverse climate gives birth to various agricultural seasons, each with its unique significance. One of these seasons is the Rabi Season, which typically falls from November to April. Rabi, which means “spring” in Arabic, is aptly named as it corresponds to the winter months when crops are sown and harvested during the spring season. This season plays a pivotal role in India’s agricultural calendar, and farmers greatly benefit from the cultivation of various crops during this time. In this blog, we will explore the significance of the Rabi season and how it benefits farmers through crop production. Key Crops of the Rabi Season The Rabi season is known for the cultivation of various crops that thrive in the cooler months. Some of the key crops grown during this season include: a. Wheat: Wheat is the most prominent Rabi crop in India and is a staple food for millions. States like Punjab, Haryana, Uttar Pradesh, and Madhya Pradesh are major wheat-producing regions. The Rabi season provides the ideal conditions for wheat cultivation, resulting in a significant boost to India's food security. b. Barley: Barley is another vital Rabi crop grown in regions with cooler climates. It is used for various purposes, including food, fodder, and the brewing industry. Rajasthan, Uttar Pradesh, and Madhya Pradesh are major barley-producing states. c. Mustard: Mustard is a key oilseed crop cultivated during the Rabi season. It provides edible oil and is grown in states like Rajasthan, Uttar Pradesh, and Madhya Pradesh. d. Gram (Chickpeas): Chickpeas are rich in protein and a popular pulse crop during the Rabi season. They are grown in states like Madhya Pradesh, Rajasthan, and Uttar Pradesh. e. Lentils: Lentils are another important pulse crop, providing a valuable source of protein. They are cultivated in various Rabi-producing states across India. Benefits to Farmers The Rabi season brings several benefits to farmers, making it a crucial part of India's agricultural calendar: a. Diversified Income: The cultivation of different crops during the Rabi season diversifies farmers' income sources. This helps reduce the risk associated with mono-cropping and provides a more stable financial foundation. b. Improved Soil Health: Rabi crops often have deep root systems, which help improve soil structure and nutrient content. This enhances the fertility of the soil for subsequent seasons. c. Water Utilization: Rabi crops typically require less water than Kharif crops, making efficient use of available water resources. This is especially important in regions with limited water availability. d. Enhanced Food Security: The production of staple crops like wheat and barley during the Rabi season contributes significantly to India's food security. It ensures a consistent supply of essential grains throughout the year. e. Livestock Feed: Some Rabi crops, like barley and chickpeas, are used as fodder for livestock, supporting the dairy and animal husbandry sectors. f. Crop Rotation: Rabi crops fit well into crop rotation cycles, reducing the risk of pests and diseases and improving overall crop health. Challenges Faced by Farmers While the Rabi season offers numerous benefits, farmers also face challenges, including: a. Weather Variability: Unpredictable weather patterns, including untimely rainfall or frost, can damage Rabi crops. b. Input Costs: The cost of inputs like seeds, fertilizers, and pesticides can be a burden on farmers, affecting their profitability. c. Market Price Fluctuations: Farmers are often vulnerable to price fluctuations in the market, which can impact their income. The Rabi season is a crucial period in India's agricultural calendar, bringing prosperity to farmers through the cultivation of various crops. It not only supports food security but also enhances soil health, diversifies income sources, and promotes efficient water use. While challenges exist, the benefits of the Rabi season make it an essential part of India's agriculture, contributing significantly to the nation's economy and the well-being of its farmers.
"""

result = summarizer(long_text)
print(result[0]["summary_text"])


Some weights of LongT5ForConditionalGeneration were not initialized from the model checkpoint at pszemraj/long-t5-tglobal-base-16384-book-summary and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Recovering a missing tied weight decoder.embed_tokens.weight from a legacy LongT5 checkpoint. Consider saving decoder.embed_tokens.weight in your checkpoint or updating the config (tie_word_embeddings=true).
Recovering a missing tied weight encoder.embed_tokens.weight from a legacy LongT5 checkpoint. Consider saving encoder.embed_tokens.weight in your checkpoint or updating the config (tie_word_embeddings=true).
Recovering a missing tied weight decoder.embed_tokens.weight from a legacy LongT5 checkpoint. Consider saving decoder.embed_tokens.weight in your checkpoint or updating the config (tie_word_embeddings=true).
Recovering a missing tied wei

In this chapter, Dr. Bhaer explains the importance of the spring season, or Rabi season, in India. It is one of the most important agricultural seasons because it brings many benefits to farmers. These include: diversified income, improved soil health, water use, and improved food security. Farmers face many challenges during the season, including weather change, price fluctuation, and pests and disease.
